In [ ]:
from google.colab import drive
import os
drive.mount('/content/drive')
path = "/content"
os.chdir(path)
os.listdir(path)

In [ ]:

!pip install -U -q kaggle
!mkdir -p ~/.kaggle
!echo '{"username":"william0617","key":"962aebdb8311ec97100f66aae69b3464"}' > ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json
 
!kaggle competitions download -c dogs-vs-cats
!unzip \*.zip


In [ ]:
import os
import shutil

train_filenames = os.listdir('train')
train_cat = filter(lambda x: x[:3] == 'cat', train_filenames)
train_dog = filter(lambda x: x[:3] == 'dog', train_filenames)
# Empty saved folders
def rmrf_mkdir(dirname):
    if os.path.exists(dirname):
        shutil.rmtree(dirname)
    os.mkdir(dirname)
# Create directories and folders
rmrf_mkdir('train2')
os.mkdir('train2/cat')
os.mkdir('train2/dog')
rmrf_mkdir('test2')
os.symlink('../test1/', 'test2/test')
for filename in train_cat:
    os.symlink('../../train/' + filename, 'train2/cat/' + filename)
for filename in train_dog:
    os.symlink('../../train/' + filename, 'train2/dog/' + filename)


In [ ]:

import h5py
from keras.models import *
from keras.layers import *
from keras.applications import *
from keras.preprocessing.image import *
from tensorflow.keras.applications.resnet50 import ResNet50


In [ ]:


# export the eigenvector
def write_gap(MODEL, image_size, lambda_func=None):
    width = image_size[0]
    height = image_size[1]
    input_tensor = Input((height, width, 3))
    x = input_tensor
    if lambda_func:
        x = Lambda(lambda_func)(x)
    base_model = MODEL(input_tensor=x, weights='imagenet', include_top=False)
    model = Model(base_model.input, GlobalAveragePooling2D()(base_model.output))
    gen = ImageDataGenerator()
    train_generator = gen.flow_from_directory("train2", image_size, shuffle=False, batch_size=16)
    test_generator = gen.flow_from_directory("test2", image_size, shuffle=False, batch_size=16, class_mode=None)
    train_generator.samples
    train = model.predict_generator(train_generator, train_generator.samples)
    test = model.predict_generator(test_generator, test_generator.samples)
    with h5py.File("gap_%s.h5" % MODEL.__name__) as h:
        h.create_dataset("train", data=train)
        h.create_dataset("test", data=test)
        h.create_dataset("label", data=train_generator.classes)

# Performs the export of eigenvector
write_gap(ResNet50, (224, 224))
# write_gap(InceptionV3, (299, 299), inception_v3.preprocess_input)
# write_gap(Xception, (299, 299), xception.preprocess_input)


In [ ]:
import numpy as np
from sklearn.utils import shuffle

# load the eigenvector
np.random.seed(2017)
X_train = []
X_test = []
for filename in ["gap_ResNet50.h5", "gap_Xception.h5", "gap_InceptionV3.h5"]:
    with h5py.File(filename, 'r') as h:
        X_train.append(np.array(h['train']))
        X_test.append(np.array(h['test']))
        y_train = np.array(h['label'])
        print( h['label'][:100])

X_train = np.concatenate(X_train, axis=1)
X_test = np.concatenate(X_test, axis=1)
X_train, y_train = shuffle(X_train, y_train)

In [ ]:
from tensorflow.python.keras.callbacks import ReduceLROnPlateau, EarlyStopping

# EarlyStopping
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    verbose=1
)

# Reduce Learning Rate
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.1,
    patience=3,
    verbose=1
)


In [ ]:
# Build the model
p_batch_size = 128
p_epochs = 1000
np.random.seed(2017)
input_tensor = Input(X_train.shape[1:])
x = Dropout(0.5)(input_tensor)
x = Dense(1, activation='sigmoid')(x)
model = Model(input_tensor, x)
model.compile(optimizer='adadelta', loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(X_train, y_train, batch_size=p_batch_size, epochs=p_epochs, validation_split=0.2,
                    callbacks=[early_stopping, reduce_lr])

In [ ]:
history

In [ ]:
import matplotlib.pyplot as plt
print(history.history.keys())
# Data Visualization
# Show training results
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(loc='best')
plt.show()
test_loss, test_acc = model.evaluate(X_train, y_train, batch_size=p_batch_size, steps=10)
print('val_loss: {:.3f}\nval_acc: {:.3f}'.format(test_loss, test_acc))


In [ ]:
y_pred = model.predict(X_test, verbose=1)
# y_pred = y_pred.clip(min=0.005, max=0.995)
test_loss, test_acc = model.evaluate(X_test, y_pred, batch_size=p_batch_size, steps=10)
print('val_loss: {:.3f}\nval_acc: {:.3f}'.format(test_loss, test_acc))


In [ ]:

import pandas as pd
from keras.preprocessing.image import *

df=pd.read_csv('sampleSubmission.csv')


gen = ImageDataGenerator()
test_generator = gen.flow_from_directory("test2", (224, 224), shuffle=False, 
                                         batch_size=16, class_mode=None)

for i, fname in enumerate(test_generator.filenames):
    index = int(fname[fname.rfind('/')+1:fname.rfind('.')])
    df.loc[i,'label']=y_pred[i]

df.to_csv('pred.csv', index=None)
df.head(10)

In [ ]:
y_pred.shape


In [ ]:
df.shape